In [1]:
import numpy as np
import plotly as px
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy
from power import *
from power_flow import *
from power.systems import *
from optimal_power_flow.studies.ac_econ_dispatch import ACEconDispatch
from optimal_power_flow.studies.ac_min_deviation import ACMinDeviation
from optimal_power_flow.studies.dc_iterative_loss import OPFIterativeLoss
from optimal_power_flow.core.ac_physics import OPFAC
from optimal_power_flow.core.dc_physics import OPFDC
from trabalhos_transmissao.utils.load_scen import apply_load_scen
from trabalhos_transmissao.utils.wnd_scen import apply_wnd_scen

In [2]:
def check_violations(net: Network, tol=1e-4):
    violations = {
        "v_mag": [],
        "line_flow": [],
    }
    n = net
    
    for line in n.lines:
        flow_out = line.p_flow_out_pu
        flow_in = line.p_flow_in_pu
        flow_max = line.flow_max_pu
        if abs(flow_out) > flow_max + tol or abs(flow_in) > flow_max + tol:
            violations["line_flow"].append((line.id, flow_out, flow_in, flow_max))
            print(f"Line {line.id} flow violation: out={flow_out}, in={flow_in}, max={flow_max}")
    for bus in n.buses:
        v_mag = bus.v_pu
        if v_mag < bus.v_min_pu - tol or v_mag > bus.v_max_pu + tol:
            violations["v_mag"].append((bus.id, v_mag, bus.v_min_pu, bus.v_max_pu))
            print(f"Bus {bus.id} voltage violation: v={v_mag}, min={bus.v_min_pu}, max={bus.v_max_pu}")
    
    has_violations = any(len(v) > 0 for v in violations.values())
    return has_violations, violations

In [3]:

NET = B6L8Charged()  # Carrega sistema de teste
H = 24 # Horizonte de Simulação (horas)
RNG1 = np.random.default_rng(seed=42)
RNG2 = np.random.default_rng(seed=41)
load_profile_base = np.array([
    0.70, 0.65, 0.62, 0.60, 0.65, 0.75, # 00:00 - 05:00 (Madrugada)
    0.85, 0.95, 1.00, 1.05, 1.10, 1.08, # 06:00 - 11:00 (Manhã/Almoço)
    1.05, 1.02, 1.00, 0.98, 1.05, 1.15, # 12:00 - 17:00 (Tarde)
    1.20, 1.18, 1.10, 1.00, 0.90, 0.80  # 18:00 - 23:00 (Noite/Pico)
])

# Perfil de Vento (Normalizado: 1.0 é a p_max_pu nominal)
# Vento costuma ser mais forte à noite e de madrugada, e cair durante o dia
wind_profile_base = np.array([
    0.90, 0.95, 0.98, 0.92, 0.85, 0.80, # Madrugada estável
    0.70, 0.60, 0.45, 0.30, 0.25, 0.35, # Queda matinal
    0.40, 0.30, 0.25, 0.35, 0.45, 0.55, # Tarde instável
    0.65, 0.75, 0.80, 0.85, 0.88, 0.92  # Recuperação noturna
])

In [4]:
# === INICIALIZAÇÃO DAS LISTAS ===

flows_out_nr, flows_out_dc_corr, flows_out_ac = [], [], []
tensions_nr, tensions_dc_corr, tensions_ac = [], [], [] 
shed_dc, shed_dc_corr, shed_ac = [], [], []
curt_dc, curt_dc_corr, curt_ac = [], [], []
fobs_dc, fobs_dc_corr, fobs_ac = [], [], []
load_shed_nr, wnd_curt_nr = [], []

# Inicializa Solvers (mantido)
net_sim_ac = deepcopy(NET)
opf_ac = ACEconDispatch(net_sim_ac)
net_sim_dc = deepcopy(NET)
opf_dc = OPFIterativeLoss(net_sim_dc)
net_sim_dc_corr = deepcopy(NET)
opf_dc_corr = OPFIterativeLoss(net_sim_dc_corr)

load_names = [l.name for l in NET.loads]
wind_names = [w.name for w in NET.wind_generators]
bus_names  = [b.name for b in NET.buses] # Para o eixo X do gráfico

print("Iniciando Simulação...")
for h in range(H):
    print(f"Hora {h+1}/{H}...")
    # --- 1. Atualização dos Cenários ---
    for l_base, l_dc, l_dc_corr, l_ac in zip(NET.loads, net_sim_dc.loads, net_sim_dc_corr.loads, net_sim_ac.loads):
        new_p = load_profile_base[h] * l_base.p_pu
        l_dc.p_pu = new_p; l_dc_corr.p_pu = new_p; l_ac.p_pu = new_p

    for w_base, w_dc, w_dc_corr, w_ac in zip(NET.wind_generators, net_sim_dc.wind_generators, net_sim_dc_corr.wind_generators, net_sim_ac.wind_generators):
        new_wnd_max = wind_profile_base[h] * w_base.p_max_pu
        w_dc.p_max_pu = new_wnd_max; w_dc_corr.p_max_pu = new_wnd_max; w_ac.p_max_pu = new_wnd_max
    
    opf_dc.update_wind_params(); opf_dc.update_load_params()
    opf_dc_corr.update_wind_params(); opf_dc_corr.update_load_params()
    opf_ac.update_wind_params(); opf_ac.update_load_params()

    # --- 2. Solvers Principais ---
    res_dc = opf_dc.solve(verbose=False); opf_dc.update_network_with_results()
    res_dc_corr = opf_dc_corr.solve(verbose=False); opf_dc_corr.update_network_with_results()
    res_ac = opf_ac.solve(verbose=False); opf_ac.update_network_with_results()

    # --- 3. Coleta AC Full ---
    tensions_ac.append([b.v_pu for b in net_sim_ac.buses])
    flows_out_ac.append([l.p_flow_out_pu for l in net_sim_ac.lines]) # <--- COLETA FLUXO AC
    
    # --- 4. Lógica de Correção ---
    pf = AC_PF(net_sim_dc_corr)
    pf.solve(verbose=False)
    pf.update_network_with_results()
    
    # Coleta estado NR (Violado)
    tensions_nr.append([b.v_pu for b in net_sim_dc_corr.buses])
    flows_out_nr.append([l.p_flow_out_pu for l in net_sim_dc_corr.lines]) # <--- COLETA FLUXO NR
    
    has_violations, violations = check_violations(net_sim_dc_corr)

    if has_violations:
        min_dev = ACMinDeviation(net_sim_dc_corr)
        res_dc_corr = min_dev.solve(verbose=False)
        min_dev.update_network_with_results()
        
    # --- 5. Coleta DC Corrigido (Final) ---
    tensions_dc_corr.append([b.v_pu for b in net_sim_dc_corr.buses])
    flows_out_dc_corr.append([l.p_flow_out_pu for l in net_sim_dc_corr.lines])


Iniciando Simulação...
Hora 1/24...
Converged in 3 iterations.
Line 1 flow violation: out=0.24503364625827795, in=-0.24313057166887542, max=0.15
Line 3 flow violation: out=-0.1468617286212326, in=0.1483872847129053, max=0.1
Line 6 flow violation: out=0.394548193842291, in=-0.3891189463713516, max=0.3
Line 7 flow violation: out=0.36229143756482673, in=-0.3598185496734567, max=0.3
Bus 3 voltage violation: v=0.9343559826028288, min=0.95, max=1.05
Bus 5 voltage violation: v=0.9498670137589649, min=0.95, max=1.05
Bus 6 voltage violation: v=0.9165306262919201, min=0.95, max=1.05
Hora 2/24...
Converged in 2 iterations.
Line 1 flow violation: out=0.2212343401672462, in=-0.2207558805947203, max=0.15
Line 3 flow violation: out=-0.10940894796456213, in=0.11137789339690496, max=0.1
Line 6 flow violation: out=0.3677393695997295, in=-0.3630906301447847, max=0.3
Line 7 flow violation: out=0.3255468760853076, in=-0.3245171362397342, max=0.3
Hora 3/24...
Converged in 2 iterations.
Line 1 flow violation

In [7]:
# Define a hora de interesse
h_plot = 2
# Garante que os dados estão em formato numpy para indexação fácil
# (Caso já não estejam)
v_nr   = np.array(tensions_nr)
v_corr = np.array(tensions_dc_corr)
v_ac   = np.array(tensions_ac)

# Nomes das barras para o eixo X
bus_names = [b.name for b in NET.buses]

fig = go.Figure()

# --- 1. NR (Estado Violado/Inicial) ---
fig.add_trace(go.Scatter(
    x=bus_names, 
    y=v_nr[h_plot], 
    mode='lines+markers', 
    name='NR (Pós-DC)',
    line=dict(color='blue', width=2, dash='dash'),
    marker=dict(size=8)
))

# --- 2. DC Corrigido (Sua Solução) ---
fig.add_trace(go.Scatter(
    x=bus_names, 
    y=v_corr[h_plot], 
    mode='lines+markers', 
    name='DC Corrigido (MinDev)',
    line=dict(color='green', width=3),
    marker=dict(size=10, symbol='diamond')
))

# --- 3. AC OPF (Referência) ---
fig.add_trace(go.Scatter(
    x=bus_names, 
    y=v_ac[h_plot], 
    mode='lines+markers', 
    name='AC OPF (Reference)',
    line=dict(color='orange', width=2),
    marker=dict(size=6)
))

# --- Linhas de Limite (0.95 e 1.05) ---
fig.add_hline(y=0.95, line_dash="dot", line_color="red", annotation_text="Min (0.95)", annotation_position="bottom right")
fig.add_hline(y=1.05, line_dash="dot", line_color="red", annotation_text="Max (1.05)", annotation_position="top right")

# --- Layout ---
fig.update_layout(
    title=f"Perfil de Tensão por Barra - Hora {h_plot}",
    xaxis_title="Barras do Sistema",
    yaxis_title="Magnitude da Tensão (pu)",
    width=1000,
    height=500,
    yaxis=dict(range=[0.85, 1.1]), # Fixa o zoom para ver bem as violações
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.8)'),
    template="plotly_white",
    hovermode="x unified" # Facilita comparar os 3 valores passando o mouse
)

fig.show()

In [8]:
# --- 1. Configuração e Tratamento de Dados ---
h_plot = 2  # Escolha a hora que quiser analisar

# Recupera os nomes e os limites INDIVIDUAIS de cada linha
line_names = [l.name for l in NET.lines]
limits     = np.array([l.flow_max_pu for l in NET.lines])

# Garante que os dados de fluxo estão em numpy array para indexação [h_plot]
f_nr   = np.array(flows_out_nr)
f_corr = np.array(flows_out_dc_corr)
f_ac   = np.array(flows_out_ac)

fig = go.Figure()

# --- 2. Plota os Limites (O "Envelope") ---
# Limite Superior (P_max)
fig.add_trace(go.Scatter(
    x=line_names, 
    y=limits,
    mode='lines', 
    name='Fluxo Máximo',
    line=dict(color='red', width=1, dash='dot'), # Linha pontilhada vermelha
    hoverinfo='y+name'
))

# Limite Inferior (-P_max)
fig.add_trace(go.Scatter(
    x=line_names, 
    y=-limits,
    mode='lines', 
    name='Fluxo Mínimo',
    line=dict(color='red', width=1, dash='dot'),
    showlegend=False, # Não precisa poluir a legenda duas vezes
    hoverinfo='skip'
))

# --- 3. Plota os Fluxos dos Métodos ---

# NR (Estado Inicial/Violado)
fig.add_trace(go.Scatter(
    x=line_names, 
    y=f_nr[h_plot],
    mode='lines+markers', 
    name='NR (Pós-DC)',
    line=dict(color='blue', width=2, dash='dash'),
    marker=dict(symbol='circle')
))

# DC Corrigido (Sua Solução MinDeviation)
fig.add_trace(go.Scatter(
    x=line_names, 
    y=f_corr[h_plot],
    mode='lines+markers', 
    name='DC Corrigido',
    line=dict(color='green', width=3),
    marker=dict(size=9, symbol='diamond')
))

# AC OPF (Referência Ideal)
fig.add_trace(go.Scatter(
    x=line_names, 
    y=f_ac[h_plot],
    mode='lines+markers', 
    name='AC OPF',
    line=dict(color='orange', width=2),
    marker=dict(symbol='x')
))

# --- 4. Layout Quadrado e Limpo ---
fig.update_layout(
    title=f"Fluxo nas Linhas vs Capacidade - Hora {h_plot}",
    xaxis_title="Linhas de Transmissão",
    yaxis_title="Fluxo de Potência Ativa (pu)",
    
    # Deixa o gráfico quadrado
    width=1000,
    height=500,
    autosize=False,
    
    template="plotly_white",
    hovermode="x unified", # Mostra todos os valores ao passar o mouse numa linha
    legend=dict(
        x=0.01, y=0.99, 
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor="Black",
        borderwidth=1
    )
)

fig.show()